In [ ]:
###############################################################################
import aliases

In [ ]:
import functools as _functools
from collections import abc as _collabc

from everest.utilities.classtools import add_defer_meths as _add_defer_meths

from everest.ptolemaic.primitive import Primitive as _Primitive
from everest.ptolemaic.ptolemaic import Ptolemaic as _Ptolemaic
from everest.ptolemaic.sprite import Sprite as _Sprite
from everest.ptolemaic.chora import Chora as _Chora, Incisable as _Incisable
from everest.ptolemaic.resourcers import Resourcer as _Resourcer
from everest.ptolemaic import intt as _intt

In [ ]:
def pass_fn(arg):
    return arg


class Armature(_Sprite):
    '''Enfolds other `Ptolemaic` objects to add functionality or context.'''

    _ptolemaic_knowntypes__ = (_Ptolemaic,)

    class Registrar:

        def process_param(self, arg, /):
            if _Resourcer.can_convert(arg):
                return _Resourcer(arg)
            return arg


class Fixture(Armature):
    '''An `Armature` that enfolds a single object, its 'content'.'''

    _req_slots__ = ('content',)

    def __init__(self, content, /):
        self.content = content
        super().__init__()


class Utiliser(Fixture):
    '''
    A `Fixture` that presents access to one of its content's methods,
    optionally with partially bound arguments.
    '''

    _req_slots__ = ('meth',)

    def __init__(self, content, meth: str, /, *args, **kwargs):
        super().__init__(content)
        meth = getattr(content, meth)
        if args or kwargs:
            meth = _functools.partial(meth, *args, **kwargs)
        self.meth = meth

    def __call__(self, /, *args, **kwargs):
        return self.meth(*args, **kwargs)


@_add_defer_meths(
    'content',
    '__iter__', '__len__', 'index', 'count',
    retrieve='__getitem__',
    )
class Brace(Fixture, _Incisable):
    '''An `Armature` that behaves like a `tuple`.'''

    Chora = _intt.InttRange

    @classmethod
    def parameterise(cls, register, arg, /):
        register(*arg)

    def _get_chora_params(self, /):
        return super()._get_chora_params(len(self.content))

    def __init__(self, /, *args):
        super().__init__(args)

    def incise(self, incisor, /):
        return type(self)(map(self.__getitem__, incisor))

    def __reversed__(self, /):
        return self[::-1]

    def __instancecheck__(self, val, /):
        return self.content.__contains__(val)


@_add_defer_meths('content', __contains__='__call__')
class Sett(_Chora, Fixture):
    '''
    A `Chora` that returns the incisor
    if a given callable called on it returns `True`.
    '''

    def _retrieve_contains_(self, incisor: object, /) -> object:
        if self.__contains__(incisor):
            return incisor
        super()._retrieve_contains_(incisor)


@_add_defer_meths(
    'content',
    '__iter__', '__len__', '__contains__', 'items', 'get',
    retrieve='__getitem__',
    )
class Mapp(Fixture, _Incisable):

    Chora = Sett

    _req_slots__ = ('_keys', '_values')

    @classmethod
    def parameterise(cls, register, arg=NotImplemented, /, **kwargs):
        if arg is NotImplemented:
            arg = kwargs
        elif kwargs:
            raise ValueError(
                f"Cannot parameterise {cls} with both args and kwargs."
                )
        if not isinstance(arg, _collabc.Mapping):
            arg = dict(arg)
        register(arg.keys(), arg.values())

    def _get_chora_params(self, /):
        return super()._get_chora_params(
            Utiliser(self._keys, '__instancecheck__')
            )

    def __init__(self, keys, values, /):
        self._keys, self._values = keys, values
        super().__init__(dict(zip(keys, values)))

    def keys(self, /):
        return self._keys

    def values(self, /):
        return self._values


Armature.PROCESSORS.update({
    _collabc.Mapping.__instancecheck__: Mapp,
    _collabc.Iterable.__instancecheck__: Brace,
    })

In [ ]:
import string

In [ ]:
mymapp = Mapp(enumerate(string.ascii_lowercase))

In [ ]:
%%timeit
mymapp[0]

In [ ]:
mydict = dict(enumerate(string.ascii_lowercase))

In [ ]:
%%timeit
mydict[0]

In [ ]:
mybrace = Brace(string.ascii_lowercase)

In [ ]:
%%timeit
mybrace[0]

In [ ]:
%%timeit
mybrace.chora.retrieve_contains(0)

In [ ]:
meth = mybrace.chora._retrieve_contains_

In [ ]:
%%timeit
meth(0)

In [ ]:
%%timeit
mybrace.chora._rangeobj[0]

In [ ]:
rangeobj = mybrace.chora._rangeobj

In [ ]:
%%timeit
rangeobj[0]

In [ ]:
tupobj = mybrace.chora._tupobj

In [ ]:
%%timeit
tupobj[0]

In [ ]:
subbrace = mybrace[-13:-5][::-2]
subbrace

In [ ]:
mysett = Sett(Utiliser(mybrace, '__instancecheck__'))

In [ ]:
mysett['d']

In [ ]:
Utiliser(subbrace, '__instancecheck__')('u')

In [ ]:
isinstance('u', subbrace)

In [ ]:
reversed(subbrace)

In [ ]:
mybrace[-1]

In [ ]:
tuple.count('a')

In [ ]:
mybrace.count('a')

In [ ]:
reversed

In [ ]:
reversed(mybrace)

In [ ]:
from everest.ptolemaic.ptolemaic import Ptolemaic as _Ptolemaic
from everest.ptolemaic.sprite import Sprite as _Sprite
from everest.ptolemaic.resourcers import Resourcer as _Resourcer
from everest.ptolemaic import collections as _collections
from everest.ptolemaic.intt import Intt as _Intt

In [ ]:
from everest.ptolemaic.collections import Collection

In [ ]:
from everest.ptolemaic import intt

In [ ]:
from everest.ptolemaic.compound import Compound as _Compound

In [ ]:
from everest.ptolemaic.compound import Compound as _Compound
from everest.ptolemaic.chora import Incisable

In [ ]:
class Armature


class Brace(Incisable, C):

    Chora = InttRange

    

In [ ]:
myobj = MyPtol()

In [ ]:
myobj[10:20]

In [ ]:
isinstance(1, Intt)

In [ ]:
from everest.ptolemaic.ptolemaic import Ptolemaic

In [ ]:
isinstance(1, Ptolemaic)

In [ ]:
from everest.ptolemaic.ptolemaic import Ptolemaic as _Ptolemaic
from everest.ptolemaic.chora import (
    Sliceable as _Sliceable, Incisable as _Incisable
    )
from everest.ptolemaic.chora import NotNone as _NotNone

class MyChora(_Sliceable):

    def _retrieve_contains_(self, incisor: int, /) -> int:
        return incisor**2

    def _incise_slice_(self,
            start: _NotNone, stop: _NotNone, step: _NotNone, /
            ):
        return (start, stop, step)


class MyPtol(_Ptolemaic, _Incisable):

    Chora = MyChora

    def incise(self, incisor, /):
        return f"Incised {self} by {incisor}"

    def retrieve(self, incisor, /):
        return f"Retrieved {incisor} from {self}"

In [ ]:
myobj = MyPtol()

In [ ]:
myobj[2:4:6]

In [ ]:
myobj[2]

In [ ]:
myobj = MyChora()

In [ ]:
myobj.chora.__getitem__

In [ ]:
issubclass(int, NotNone)

In [ ]:
issubclass(NotNone, int)

In [ ]:
class MyClass:

    @classmethod
    def __subclasscheck__(cls, other, /):
        print('hello')

In [ ]:
issubclass(int, MyClass)

In [ ]:
issubclass(int, NotNone)

In [ ]:
import abc as _abc


class Null(_abc.ABC):

    @classmethod
    def __subclasscheck__(cls, arg, /):
        return True

In [ ]:
isinstance(1, Null)

In [ ]:
issubclass(int, NotNone)

In [ ]:
myobj[2:3:4]

In [ ]:
class MyClass:

    def __getitem__(self, arg, /):
        '''Placeholder.'''
        print('Foo!')


def __getitem__(self, arg, /):
    return arg**2


myobj = MyClass()
myobj.__getitem__ = __getitem__
myobj[2]